Imports needen only in Colab

In [ ]:
# !pip install -q tensorflow-gpu==2.0.0-beta0

# import os
# from google.colab import drive
# drive.mount('/content/drive/')
# try:
#     os.chdir("drive/My Drive/snr/project")
# except FileNotFoundError:
#     pass

All other imports

In [ ]:
import tensorflow as tf
from tensorflow import keras

from definitions import *
from preprocessing import load_datasets, time_footprint

# Load datasets

In [ ]:
train_ds, test_ds, val_ds = load_datasets()

# Create model

In [ ]:
ALPHA = 1.0  # controls the width of the network
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                               include_top = False, 
                                               alpha       = ALPHA,
                                               weights     = 'imagenet')

Now we want to train all layers of **MobileNetV2** model.

In [ ]:
base_model.summary()

In [ ]:
base_model.trainable = True

In [ ]:
model = tf.keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(NUMBER_OF_LABELS, activation='softmax')
])

In [ ]:
base_learning_rate = 5e-6
model.compile(optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate), 
              loss      = 'sparse_categorical_crossentropy', 
              metrics   = ['accuracy'])

In [ ]:
model.summary()

# Train model

In [ ]:
epochs          = 150
steps_per_epoch = TRAIN_FILES//BATCH_SIZE
val_steps       =   VAL_FILES//BATCH_SIZE

time_fp = time_footprint()
log_dir = os.path.join(LOG_DIR, 'task_3_a', time_fp)
os.makedirs(log_dir)

checkpoint_path = log_dir + "/task_3_a_" + time_fp + ".ckpt"

callbacks = [
    keras.callbacks.TensorBoard(log_dir=log_dir),
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                       save_weights_only=True,
                                       verbose=1,
                                       save_best_only=True,
                                       monitor='val_loss')
]


history = model.fit(train_ds,
                    epochs           = epochs,
                    steps_per_epoch  = steps_per_epoch,
                    validation_data  = val_ds, 
                    validation_steps = val_steps,
                    callbacks        = callbacks)


val_acc = history.history['val_accuracy'][0]

model_name = "task_3_a_" + time_fp + '_acc_' + f'{val_acc:.4f}' + '.h5'
model.save(MODEL_DIR + '/' + model_name)

## Create simpler model

In [ ]:
simpler_base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                                       include_top = False, 
                                                       alpha       = ALPHA,
                                                       weights     = 'imagenet')

Remove block_16 layers from base model

In [ ]:
last_layer_old = simpler_base_model.get_layer('block_15_project_BN').output
x = keras.layers.Conv2D(**simpler_base_model.get_layer('Conv_1').get_config())(last_layer_old)
x = keras.layers.BatchNormalization(**simpler_base_model.get_layer('Conv_1_bn').get_config())(x)
last_layer_new = keras.layers.ReLU(**simpler_base_model.get_layer('out_relu').get_config())(x)

In [ ]:
simpler_base_model = tf.keras.Model(inputs = simpler_base_model.layers[0].input, outputs = last_layer_new, name='Simpler')

In [ ]:
simpler_base_model.summary()

In [ ]:
simpler_base_model.trainable = True

In [ ]:
simpler_model = tf.keras.Sequential([
    simpler_base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(NUMBER_OF_LABELS, activation='softmax')
])

In [ ]:
base_learning_rate = 5e-6
simpler_model.compile(optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate), 
                      loss      = 'sparse_categorical_crossentropy', 
                      metrics   = ['accuracy'])

In [ ]:
simpler_model.summary()

# Train simpler model

In [ ]:
epochs          = 150
steps_per_epoch = TRAIN_FILES//BATCH_SIZE
val_steps       =   VAL_FILES//BATCH_SIZE

time_fp = time_footprint()
log_dir = os.path.join(LOG_DIR, 'task_3_b', time_fp)
os.makedirs(log_dir)

checkpoint_path = log_dir + "/task_3_b_" + time_fp + ".ckpt"

callbacks = [
    keras.callbacks.TensorBoard(log_dir=log_dir),
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                       save_weights_only=True,
                                       verbose=1,
                                       save_best_only=True,
                                       monitor='val_loss')
]


history = simpler_model.fit(train_ds,
                            epochs           = epochs,
                            steps_per_epoch  = steps_per_epoch,
                            validation_data  = val_ds, 
                            validation_steps = val_steps,
                            callbacks        = callbacks)


val_acc = history.history['val_accuracy'][0]

model_name = "task_3_b_" + time_fp + '_acc_' + f'{val_acc:.4f}' + '.h5'
model.save(MODEL_DIR + '/' + model_name)